This notebook shows how to use and test the script I made for converting sbml files to stan files.

## How to run the sbml-to-stan script
First let's check that there is an appropriate sbml file at the relative path `../data_in/t_brucei.xml`

In [1]:
!head -10 ../data_in/t_brucei.xml

It looks like that file exists and contains the string 'SBML'! Now we can run the script:

In [2]:
%run convert_sbml_to_stan.py --input_file ../data_in/t_brucei.xml

Parsing sbml input...
Converting parsed sbml to Stan...
Writing Stan code to /Users/tedgro/Code/ecoli_stan/python/../stan/autogen/t_brucei.stan...
Finished!


## Quickly sanity checking the output

It (says that it) worked! Let's have a quick look at the output to see if it looks like it should:

In [3]:
!head -10 ../stan/autogen/t_brucei.stan
print('...')
!tail -42 ../stan/autogen/t_brucei.stan

real Function_for_Glucose_transport(real GlcE,real GlcI,real K1Glc,real Vm1,real Vt,real afac,real tot_cell){
  return tot_cell / Vt * Vm1 * (GlcE - GlcI) / (K1Glc + GlcE + GlcI + afac * GlcE * GlcI / K1Glc);
}

real Function_for_Hexokinase(real ADPg,real ATPg,real Glc6P,real GlcI,real K2ADPg,real K2ATPg,real K2Glc6P,real K2GlcI,real Vm2,real Vt,real tot_cell){
  return tot_cell / Vt * Vm2 * GlcI * ATPg / (K2ATPg * K2GlcI * (1 + Glc6P / K2Glc6P + GlcI / K2GlcI) * (1 + ATPg / K2ATPg + ADPg / K2ADPg));
}

real Function_for_Glucose_phosphate_isomerase(real Fru6P,real Glc6P,real K3Fru6P,real K3Glc6P,real Vm3,real Vt,real glycosome,real tot_cell){
  return tot_cell / Vt * Vm3 * (Glc6P / K3Glc6P - Fru6P / K3Fru6P) / (1 + Glc6P / K3Glc6P + Fru6P / K3Fru6P) / glycosome;
...

vector get_odes(vector fluxes){
  real vGlcTr = fluxes[1];
  real vHK = fluxes[2];
  real vPGI = fluxes[3];
  real vPFK = fluxes[4];
  real vALD = fluxes[5];
  real vTPI = fluxes[6];
  real vGAPdh = fluxes[7];
  real vGDH 

This looks roughly correct: at the top of the file there are some functions for finding reaction rates given parameters, and at the bottom there is a function calcluating rates of change of some metabolites, and most importantly a `steady_state_equation` function with the right functional form to use in Stan's equation solver.

# Generating a time course with the auto-generated Stan code

One way to test if the Stan code really copies the sbml file is to run a time course and compare the output with some numbers generated independently from the same sbml.

The required time course can be generated using the following Stan model:

In [4]:
!cat ../stan/timecourse_model_template.stan

functions {
#include REPLACE_THIS_WORD
  real[] ode(real t,        // time
             real[] s,      // state
             real[] theta,  // parameters
             real[] x_r,    // data (real)
             int[] x_i){   // data (integer)
    return to_array_1d(steady_state_equation(to_vector(s), to_vector(theta), x_r, x_i));
  }
}
data {
  int<lower=1> N_ode;
  int<lower=1> N_derived;
  int<lower=1> N_known_real;
  int<lower=1> P;
  int<lower=1> T;
  real initial_metabolite_ode[N_ode];
  real kinetic_parameters[P];
  real known_reals[N_known_real];
  real ts[T];
  real t0;
}
generated quantities {
  int known_ints[0];
  real ode_metabolite_sim[T+1,N_ode]; 
  real derived_quantity_sim[T+1, N_derived];
  ode_metabolite_sim[1] = initial_metabolite_ode;
  ode_metabolite_sim[2:T+1] = integrate_ode_rk45(ode,
                                                 initial_metabolite_ode,
                                                 t0,
                                                 ts,
   

The python script below replaces the word `REPLACE_THIS_WORD` above with the relative path to `t_brucei.stan`, then compiles the model and runs it using the initial concentrations and parameter values from `t_brucei.xml`. The results are written to `data_out/timecourse_t_brucei.csv`.

In [5]:
!python test_timecourse.py

Using cached StanModel
Iteration: 1 / 1 [100%]  (Sampling)

 Elapsed Time: 0 seconds (Warm-up)
               0.000432 seconds (Sampling)
               0.000432 seconds (Total)

/Users/tedgro/.pyenv/versions/3.7.2/lib/python3.7/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
Writing results to ../data_out/timecourse_t_brucei.csv...
Finished!


In [7]:
import pandas as pd

pd.read_csv('../data_out/timecourse_t_brucei.csv').head()

,sim_time,BPGA13,DHAP,Fru16BP,Fru6P,GAP,Glc6P,GlcE,GlcI,Gly,...,ATPg,DHAPc,DHAPg,Gly3P,Gly3Pc,Gly3Pg,PEPc,PGAg,Vc,Vg
0,0.0,0.032675,3.89921,16.5371,0.511773,0.039933,2.07199,5.0,0.034001,0.0,...,2.61685,4.015343,1.309058,0.956179,0.984657,0.321012,0.840863,0.671134,5.4554,0.2446
1,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4554,0.2446
2,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4554,0.2446
3,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4554,0.2446
4,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4554,0.2446


Uh oh! The model failed to generate any numbers after the first iteration - something must have gone wrong.